In [1]:
import json
import zipfile
import os
import pandas as pd
from matplotlib import pyplot
from math import sqrt 
import numpy as np 
import scipy.misc 
from IPython.display import display 
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator 
from keras.utils import plot_model
from keras.models import Model, Sequential
from keras.layers import Input, Dense, Flatten, Dropout, BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import concatenate
from keras.optimizers import Adam, SGD
from keras.regularizers import l1, l2
from keras.utils import plot_model
from matplotlib import pyplot as plt
from sklearn.metrics import confusion_matrix
import itertools
from pylab import rcParams

%matplotlib inline


Using TensorFlow backend.


In [2]:
data= pd.read_csv(r"C:\Users\ee527\Desktop\train.csv")
data.head()

,emotion,pixels
0,0,70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...
1,0,151 150 147 155 148 133 111 140 170 174 182 15...
2,2,231 212 156 164 174 138 161 173 182 200 106 38...
3,4,24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...
4,6,4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...


In [3]:
rcParams['figure.figsize'] = 15, 10

checking the number of each class in our dataset

In [4]:
data.emotion.value_counts()

3    7215
6    4965
4    4830
2    4097
0    3995
5    3171
1     436
Name: emotion, dtype: int64

In [5]:
num_classes = 7
width = 48
height = 48
emotion_labels = ["Angry", "Disgust", "Fear", "Happy", "Sad", "Surprise", "Neutral"]
classes=np.array(emotion_labels)

In [6]:
depth = 1
height = int(sqrt(len(data.pixels[0].split()))) 
width = int(height)

In [8]:

def gray_to_rgb(im):
  '''
  converts images from single channel images to 3 channels
  '''

  w, h = im.shape
  ret = np.empty((w, h, 3), dtype=np.uint8)
  ret[:, :, 2] =  ret[:, :, 1] =  ret[:, :, 0] =  im
  return ret

def convert_to_image(pixels, mode="save", t="gray"):
  
  '''
  convert the input pixels from the single string row to  48*48 array with real pixel values
  when mode = "save" it keeps the images in flat array shape, otherwise it converts it to 48*48
  when t (for type) = "gray, it keeps the pixels single channel, otherwise it converts it to 3 channels
  '''

  if type(pixels) == str:
      pixels = np.array([int(i) for i in pixels.split()])
  if mode == "show":
    if t == "gray":
      return pixels.reshape(48,48)
    else:
      return gray_to_rgb(pixels.reshape(48,48))
  else:
      return pixels

data["pixels"] = data["pixels"].apply(lambda x : convert_to_image(x, mode="show", t="gray"))
from sklearn.model_selection import train_test_split
#split the data to train, test, and validation
X_train, X_test, y_train, y_test = train_test_split(data["pixels"],  data["emotion"], test_size=0.2, random_state=1)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=1)

X_train = np.array(list(X_train[:]), dtype=np.float)
X_val = np.array(list(X_val[:]), dtype=np.float)
X_test = np.array(list(X_test[:]), dtype=np.float)

y_train = np.array(list(y_train[:]), dtype=np.float)
y_val = np.array(list(y_val[:]), dtype=np.float)
y_test = np.array(list(y_test[:]), dtype=np.float)

X_train = X_train.reshape(X_train.shape[0], 48, 48, 1) 
X_val = X_val.reshape(X_val.shape[0], 48, 48, 1)
X_test = X_test.reshape(X_test.shape[0], 48, 48, 1)

In [9]:
num_train = X_train.shape[0]
num_val = X_val.shape[0]
num_test = X_test.shape[0]

In [10]:
y_train = np_utils.to_categorical(y_train, num_classes) 
# y_val = val_set.emotion 
y_val = np_utils.to_categorical(y_val, num_classes) 
# y_test = test_set.emotion 
y_test = np_utils.to_categorical(y_test, num_classes) 

In [11]:
datagen = ImageDataGenerator( 
    rescale=1./255,
    rotation_range = 10,
    horizontal_flip = True,
    width_shift_range=0.1,
    height_shift_range=0.1,
    fill_mode = 'nearest')

testgen = ImageDataGenerator( 
    rescale=1./255
    )
datagen.fit(X_train)
batch_size = 64

In [ ]:
for X_batch, y_batch in datagen.flow(X_train, y_train, batch_size=9):
    for i in range(0, 9): 
        pyplot.axis('off') 
        pyplot.subplot(330 + 1 + i) 
        # print(np.where(y_batch[i] == 1)[0][0])
        pyplot.title(emotion_labels[np.where(y_batch[i] == 1)[0][0]])
        pyplot.imshow(X_batch[i].reshape(48, 48), cmap=pyplot.get_cmap('gray'))
    pyplot.axis('off') 
    pyplot.show() 
    break 

In [14]:
import keras
train_flow = datagen.flow(X_train, y_train, batch_size=batch_size) 
val_flow = testgen.flow(X_val, y_val, batch_size=batch_size) 
test_flow = testgen.flow(X_test, y_test, batch_size=batch_size) 

In [ ]:
 def conv_bn_relu(x, filters, kernel_size, strides):
    x = tf.keras.layers.Conv2D(filters=filters, kernel_size=kernel_size, strides=strides, padding='same')(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.ReLU()(x)
    return x

def inverted_residual_block(x, filters, strides, expansion_factor):
    in_channels = x.shape[-1]
    # pointwise convolution 1x1
    x = conv_bn_relu(x, filters=expansion_factor*in_channels, kernel_size=1, strides=1)
    # depthwise convolution 3x3
    x = tf.keras.layers.DepthwiseConv2D(kernel_size=3, strides=strides, padding='same')(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.ReLU()(x)
    # pointwise convolution 1x1
    x = tf.keras.layers.Conv2D(filters=filters, kernel_size=1, strides=1, padding='same')(x)
    x = tf.keras.layers.BatchNormalization()(x)
    # skip connection
    if strides == 1 and in_channels == filters:
        x = tf.keras.layers.Add()([x, x])
    return x

def mobilenet_v2(input_shape, num_classes):
    inputs = tf.keras.layers.Input(shape=input_shape)
    x = conv_bn_relu(inputs, filters=32, kernel_size=3, strides=2)
    x = inverted_residual_block(x, filters=16, strides=1, expansion_factor=1)
    x = inverted_residual_block(x, filters=24, strides=2, expansion_factor=6)
    x = inverted_residual_block(x, filters=24, strides=1, expansion_factor=6)
    x = inverted_residual_block(x, filters=32, strides=2, expansion_factor=6)
    x = inverted_residual_block(x, filters=32, strides=1, expansion_factor=6)
    x = inverted_residual_block(x, filters=32, strides=1, expansion_factor=6)
    x = inverted_residual_block(x, filters=64, strides=2, expansion_factor=6)
    x = inverted_residual_block(x, filters=64, strides=1, expansion_factor=6)
    x = inverted_residual_block(x, filters=64, strides=1, expansion_factor=6)
    x = inverted_residual_block(x, filters=64, strides=1, expansion_factor=6)
    x = inverted_residual_block(x, filters=96, strides=1, expansion_factor=6)
    x = inverted_residual_block(x, filters=96, strides=1, expansion_factor=6)
    x = inverted_residual_block(x, filters=96, strides=1, expansion_factor=6)
    x = inverted_residual_block(x, filters=160, strides=2, expansion_factor=6)
    x = inverted_residual_block(x, filters=160, strides=1, expansion_factor=6)
    x = inverted_residual_block(x, filters=160, strides=1, expansion_factor=6)
    x = inverted_residual_block(x, filters=320, strides=1, expansion_factor=6)
    x = conv_bn_relu(x, filters=1280, kernel_size=1, strides=1)
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    outputs = tf.keras.layers.Dense(num_classes, activation='softmax')(x)
    model = tf.keras.models.Model(inputs=inputs, outputs=outputs)
    return model

input_shape=(48,48,1)
conv5_model = mobilenet_v2(input_shape, 7)


In [16]:
opt = Adam(lr=0.0001, decay=1e-6)
conv5_model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy']) 

In [18]:
from keras.callbacks import ModelCheckpoint
filepath="weights_min_loss.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [ ]:
num_epochs = 100  
history = conv5_model.fit_generator(train_flow, 
                    steps_per_epoch=len(X_train) / batch_size, 
                    epochs=num_epochs,  
                    verbose=2,  
                    callbacks=callbacks_list,
                    validation_data=val_flow,  
                    validation_steps=len(X_val) / batch_size) 

In [20]:
%matplotlib inline
def visualize_acc(history):

  train_loss=history.history['loss']
  val_loss=history.history['val_loss']
  train_acc=history.history['accuracy']
  val_acc=history.history['val_accuracy']

  epochs = range(len(train_acc))

  plt.plot(epochs,train_loss,'r', label='train_loss')
  plt.plot(epochs,val_loss,'b', label='val_loss')
  plt.title('train_loss vs val_loss')
  plt.xlabel('epoch')
  plt.ylabel('loss')
  plt.legend()
  plt.figure()

  plt.plot(epochs,train_acc,'r', label='train_acc')
  plt.plot(epochs,val_acc,'b', label='val_acc')
  plt.title('train_acc vs val_acc')
  plt.xlabel('epoch')
  plt.ylabel('accuracy')
  plt.legend()
  plt.figure()

visualize_acc(history)